# Limestone addition to the cement recipe

<p class="acknowledgement">Written jountly by Svetlana Kyas (ETH Zurich) and Dan Miron (PSI) on April 4th, 2022</p>

This tutorial shows how Reaktoro can be used for realistic modeling of limestone addition to the cement formulation and the effects of this mixing.

```{note}
If your main interest is in calculating thermodynamic properties rather than modeling chemical equilibria and kinetics, you should check out [ThermoFun] (https://thermohub.org/thermofun/thermofun/), an excellent project dedicated to this task.
```

The model considered in this tutorial uses a **thermofun** database `cemdata18` based on [Cemdata](https://www.empa.ch/web/s308/thermodynamic-data), the thermodynamic data for hydrated solids in the Portland cement system (CaO-Al2O3-SiO2-CaSO4-CaCO3-Fe2O3-MgO-H2O). We start with the initialization of the chemical system by defining the content of the aqueous phase. It is to be governed by the Debye-Huckel activity model, where the parameters å and b are set for specific ionic species and the parameter b for neutral species.

In [1]:
from reaktoro import *

# Define the Thermofun database
db = ThermoFunDatabase("cemdata18")

# Define an aqueous solution phase
solution = AqueousPhase(speciate("H O K Na S Si Ca Mg Al C Cl"))

# Set up a and b parameters for the ionic species (KOH, b = 0.123, a = 3.67) and the Debye-Huckel activity model
params = ActivityModelDebyeHuckelParams()
params.aiondefault = 3.67
params.biondefault = 0.123
params.bneutraldefault = 0.123
solution.setActivityModel(ActivityModelDebyeHuckel(params))

# Define the gas phase
gaseous = GaseousPhase(speciate("H O C"))

We continue with the definition of mineral phases and solid phases. For each solid phase, we also provide the name to avoid that it is automatically generated once. We finish the creation of the chemical system by initializing it with `cemdata18` database and the phases we have defined.

In [2]:
# Define minerals phases
minerals = MineralPhases("Cal hydrotalcite Portlandite hemicarbonate monocarbonate Amor-Sl FeOOHmic Gbs Mag")

# Define the hydrogarnet solid phase
ss_C3AFS084H  = SolidPhase("C3FS0.84H4.32 C3AFS0.84H4.32")
ss_C3AFS084H.setName("ss_C3AFS084H")
# Define the ettrignite solid phase
ss_ettringite = SolidPhase("ettringite ettringite30")
ss_ettringite.setName("ss_Ettrignite")
# Define the monosulfate solid phase
ss_OH_SO4_AFm = SolidPhase("C4AH13 monosulphate12")
ss_OH_SO4_AFm.setName("ss_Monosulfate")
# Define the CSHQ solid phase
ss_CSHQ = SolidPhase("CSHQ-TobD CSHQ-TobH CSHQ-JenH CSHQ-JenD KSiOH NaSiOH")
ss_CSHQ.setName("ss_CSHQ")

# Define the chemical system by providing database, aqueous phase, minerals, and solid solutions
system = ChemicalSystem(db, solution, minerals, gaseous,
                        ss_C3AFS084H,
                        ss_ettringite,
                        ss_OH_SO4_AFm,
                        ss_CSHQ)

Next, we set up the equilibrium specifications, the equilibrium conditions, and the equilibrium solver, all of which are used for the equilibrium calculations:

In [3]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()

# Define conditions to be satisfied at chemical equilibrium
conditions = EquilibriumConditions(specs)
conditions.temperature(20.0, "celsius")
conditions.pressure(1.0, "bar")

# Define chemical and aqueous properties
props = ChemicalProps(system)
aprops = AqueousProps(system)

# Define equilibrium options
opts = EquilibriumOptions()
opts.optima.output.active = False
opts.epsilon = 1e-13

# Define the equilibrium solver
solver = EquilibriumSolver(specs)
solver.setOptions(opts)

In the following, we compile the chemical materials corresponding to:
* 100 g of the cement clinker, which consists of the various clinker phases with additives,
* 1 kg of water and
* 1 g of calcite.

In [4]:
# We define the materials for our equilibrium recipe
# Cement clinker composition from XRF as given in Lothenbach et al., (2008) recalculated for 100g
cement_clinker = Material(system)
cement_clinker.add("SiO2" , 20.47, "g")
cement_clinker.add("CaO"  , 65.70, "g")
cement_clinker.add("Al2O3",  4.90, "g")
cement_clinker.add("Fe2O3",  3.20, "g")
cement_clinker.add("K2O"  ,  0.79, "g")
cement_clinker.add("Na2O" ,  0.42, "g")
cement_clinker.add("MgO"  ,  1.80, "g")
cement_clinker.add("SO3"  ,  2.29, "g")
cement_clinker.add("CO2"  ,  0.26, "g")
cement_clinker.add("O2"   ,  0.15, "g")

# Define water
water = Material(system)
water.add("H2O", 1000.0, "g")

# Define calcite
calcite = Material(system)
calcite.add("CaCO3", 1, "g")

Next, we specify the list of phases whose volume we want to track. This list of phases is used to define the columns of the `pandas.DataFrame` instance where the volume of the phases is stored as a percentage to the total volume of the system.

In [5]:
import numpy as np
# Create list of species and phases names, list of Species objects, and auxiliary amounts array
phases_list_str = "ss_C3AFS084H ss_Ettrignite ss_Monosulfate ss_CSHQ " \
                  "Cal hydrotalcite Portlandite hemicarbonate monocarbonate Amor-Sl FeOOHmic Gbs Mag".split()
volume = np.zeros(len(phases_list_str))

# Define dataframe to collect amount of the selected species
import pandas as pd
columns = ["CaCO3"] \
          + ["volume_perc_" + name for name in phases_list_str]
df = pd.DataFrame(columns=columns)

In the following loop, we simulate the addition of calcite at the expense of clinker in the cement mixture. In these sequential calculations, the percentage of the phase volume of the selected phases is determined.

In [6]:
# Number of steps
steps_num = 19

for i in range(0, steps_num):

    # Define a cement mix of 0.5 water/binder at each step calcite is added at the expense of clinker
    cement_mix = Material(system)
    cement_mix = cement_clinker(100.0-i, "g") + calcite(i, "g") + water(50.0, "g")

    # Equilibrate cement mix
    state = cement_mix.equilibrate(20.0, "celsius", 1.0, "bar", opts)
    res = cement_mix.result()
    
    if not res.optima.succeeded:

        # Equilibrate the resulting chemical state with equilibrium solver
        res = solver.solve(state, conditions)
        if not res.optima.succeeded: continue

    # Update chemical and aqueous properties
    props.update(state)
    aprops.update(state)

    for j in range(0, len(phases_list_str)):
        # Collecting the volume of specified phase
        volume[j] = float(props.phaseProps(phases_list_str[j]).volume())
    volume_perc = volume / float(props.volume()) * 100

    # Update dataframe with obtained values
    df.loc[len(df)] = np.concatenate([[i], volume_perc])

To inspect the content of the `pandas.DataFrame`, we can just output it in the code cell:

In [7]:
df

,CaCO3,volume_perc_ss_C3AFS084H,volume_perc_ss_Ettrignite,volume_perc_ss_Monosulfate,volume_perc_ss_CSHQ,volume_perc_Cal,volume_perc_hydrotalcite,volume_perc_Portlandite,volume_perc_hemicarbonate,volume_perc_monocarbonate,volume_perc_Amor-Sl,volume_perc_FeOOHmic,volume_perc_Gbs,volume_perc_Mag
0,2.0,2.876921,3.504765,3.108167e-11,14.444411,0.322619,1.281322,8.272928,1.512770e-11,1.301151,1.541934e-12,1.824028e-12,1.699105e-12,2.367347e-12
1,3.0,2.862425,3.488307,3.125569e-11,14.379008,0.522160,1.275348,8.232058,1.521240e-11,1.297116,1.550567e-12,1.834241e-12,1.708618e-12,2.380602e-12
2,4.0,2.847779,3.471673,3.143168e-11,14.312845,0.723952,1.269307,8.190760,1.529805e-11,1.293010,1.559298e-12,1.844569e-12,1.718239e-12,2.394006e-12
3,5.0,2.832979,3.454860,3.160966e-11,14.245909,0.928032,1.263197,8.149027,1.538468e-11,1.288833,1.568127e-12,1.855014e-12,1.727968e-12,2.407562e-12
4,6.0,2.818024,3.437864,3.178968e-11,14.178186,1.134441,1.257019,8.106851,1.547230e-11,1.284582,1.577058e-12,1.865578e-12,1.737809e-12,2.421274e-12
5,7.0,2.802911,3.420682,3.197177e-11,14.109662,1.343219,1.250770,8.064227,1.556092e-11,1.280256,1.586091e-12,1.876264e-12,1.747763e-12,2.435142e-12
6,8.0,2.787637,3.403310,3.215596e-11,14.040325,1.554406,1.244449,8.021146,1.565057e-11,1.275855,1.595229e-12,1.887073e-12,1.757832e-12,2.449172e-12
7,9.0,2.772200,3.385745,3.234230e-11,13.970158,1.768045,1.238055,7.977600,1.574126e-11,1.271375,1.604473e-12,1.898008e-12,1.768018e-12,2.463364e-12
8,10.0,2.756597,3.367984,3.253081e-11,13.899147,1.984178,1.231587,7.933582,1.583301e-11,1.266816,1.613825e-12,1.909071e-12,1.778324e-12,2.477722e-12
9,11.0,2.740826,3.350022,3.272154e-11,13.827277,2.202850,1.225043,7.889084,1.592584e-11,1.262176,1.623287e-12,1.920264e-12,1.788750e-12,2.492249e-12


To visualize the distribution of different minerals in the cement recipe while the limestone addition, we us [bokeh](https://bokeh.org/) plotting library.


In [8]:
from bokeh.plotting import figure, show
from bokeh.palettes import brewer
from bokeh.io import output_notebook
output_notebook()

p = figure(
    title="EFFECT OF LIMESTONE",
    x_axis_label=r'CACO3 [%]',
    y_axis_label='PHASE VOLUME [%]',
    sizing_mode="scale_width",
    plot_height=300)

volume_names = ["Cal", "hydrotalcite", "Portlandite", "ss_CSHQ", "ss_C3AFS084H", "ss_Ettrignite", "monocarbonate"]
volume_perc_names = ["volume_perc_" + name for name in volume_names]
p.varea_stack(stackers=volume_perc_names, x='CaCO3', color=brewer['Spectral'][len(volume_names)], legend_label=volume_names, source=df)
p.legend[0].items.reverse()

show(p)

Loading BokehJS ...